# LeeRobust

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.LeeRobust)

class LeeRobust(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [3]:
model = pya.models.LeeRobust()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'leerobust'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2019
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "Lee, Yunsung, et al. \"Placental epigenetic clocks: estimating gestational age using placental DNA methylation levels.\" Aging (Albany NY) 11.12 (2019): 4238."
model.metadata["doi"] = "https://doi.org/10.18632/aging.102049"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download directly with curl

In [5]:
supplementary_url = "https://www.aging-us.com/article/102049/supplementary/SD2/0/aging-v11i12-102049-supplementary-material-SD2.csv"
supplementary_file_name = "coefficients.csv"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

## Load features

#### From CSV file

In [6]:
df = pd.read_csv('coefficients.csv')
df['feature'] = df['CpGs']
df['coefficient'] = df['Coefficient_RPC']
df = df[df.coefficient != 0]

model.features = features = df['feature'][1:].tolist()

## Load weights into base model

In [7]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

#### Linear model

In [8]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [9]:
model.reference_values = None

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [11]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'Lee, Yunsung, et al. "Placental epigenetic clocks: estimating '
             'gestational age using placental DNA methylation levels." Aging '
             '(Albany NY) 11.12 (2019): 4238.',
 'clock_name': 'leerobust',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.18632/aging.102049',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2019}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: None
postprocess_dependencies: None
features: ['cg00009871', 'cg00035630', 'cg00056066', 'cg00057476', 'cg00063979', 'cg00073090', 'cg00091483', 'cg00173659', 'cg00192031', 'cg00239899', 'cg00307685', 'cg00398130', 'cg00400547', 'cg00419702', 'cg00466827', 'cg00469856', 'cg00501482', 'cg00521434', 'cg00595223', 'cg00639010', 'cg00

## Basic test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[42.0170],
        [55.4254],
        [49.9402],
        [36.4192],
        [45.8815],
        [10.6421],
        [32.3821],
        [38.4876],
        [24.5012],
        [70.2651]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: coefficients.csv
